In [1]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [2]:
rawdata = pd.read_csv('data/milk.csv')
data = rawdata.copy()

data = rawdata.copy()

#lag auctions within 1 vendor
data = data.sort_values(['VENDOR','YEAR','MONTH','DAY','SYSTEM'])
bids = data.groupby(['VENDOR','YEAR','MONTH','DAY','SYSTEM'], as_index=False).mean()
bids = bids[['VENDOR','YEAR','MONTH','DAY','SYSTEM']]
sys_lag = bids.groupby(['VENDOR']).shift(1)
bids['VENDOR_LAG'] = sys_lag['SYSTEM']
data = pd.merge(data, bids, how='left', 
                 on=['VENDOR','YEAR','MONTH','DAY','SYSTEM'], suffixes=('', '_LAG') ) 

#lag auctions
data =data.sort_values(['YEAR','MONTH','DAY','SYSTEM'])
aucts =  data.groupby(['YEAR','MONTH','DAY','SYSTEM'], as_index=False).mean()
aucts = aucts[['YEAR','MONTH','DAY','SYSTEM']]
auct_lag = aucts.shift(1)
aucts['AUCT_LAG'] = auct_lag['SYSTEM']
data = pd.merge(data, aucts, how='left', 
                 on=['YEAR','MONTH','DAY','SYSTEM'], suffixes=('', '_LAG') ) 

prev_auct = 1.*(data['AUCT_LAG'] == data['VENDOR_LAG'])
data['PAST_AUCT'] = prev_auct

In [3]:
#general house keeping
data = data[ (data['YEAR']>=1980)]
data = data[(data['MONTH'] >= 4) & (data['MONTH'] <= 9) & (data['DAY'] !=0) ] #need data with time index
data = data[(~np.isnan(data['SCORE']) ) & (data['QSCORE']!=0 )] #need data with QWW and WW


data['COOLER'] = data['COOLER'].fillna(0)
data['ESC'] = data['ESC'].fillna(0)
data['ONEBID'] = 1.*(data['NUM'] == 1)

#various keys
milk = ['SCORE']
auct_key = ['YEAR','MONTH','DAY','SYSTEM','FMOZONE']
cts = ['FMO','GAS','POPUL','QSCORE']
dummies = ['COOLER','ESC', 'NUM'] #delete num?


#baseline stuff/logs
lmilk = ['L'+x for x in milk]
lcts = ['L'+x for x in cts]
data[lcts] = np.log(data[cts])
data[lmilk] = np.log(data[milk])

#set up lags
lags = 10
lagkeys = [l+str(i) for l in ['LSCORE_min','LSCORE_max'] for i in range(1,1+lags)]
aucts = data.groupby(auct_key, as_index=False)[milk].mean()[auct_key]

#note data is already sorted by date
data = data.sort_values(['YEAR','MONTH','DAY'])
min_lag = data.groupby(auct_key, as_index=False).min()
for t in range(1,1+lags):
    min_lagt = min_lag.shift(t)[lmilk]
    min_lagt = pd.concat((aucts, min_lagt), axis=1)
    data = pd.merge(data, min_lagt, how='left', on=auct_key, suffixes=('', '_min%s'%(t)) ) 
    
max_lag = data.groupby(auct_key, as_index=False).max()
for t in range(1,1+lags):
    max_lagt = max_lag.shift(t)[lmilk]
    max_lagt = pd.concat((aucts, max_lagt), axis=1)
    data = pd.merge(data, max_lagt, how='left', on=auct_key, suffixes=('', '_max%s'%(t)) )    

    
#set up lags but prev year 
yearlags = 3
year_lagkeys = [l+str(i) for l in ['LSCORE_miny','LSCORE_maxy'] for i in range(1,1+yearlags)]


#note data is sorted by year now
data = data.sort_values(['SYSTEM','YEAR','MONTH','DAY'])

year_min_lag = data.groupby(auct_key, as_index=False).min().sort_values(['SYSTEM','YEAR','MONTH','DAY'])
for t in range(1,1+lags):
    min_lagt = year_min_lag.groupby(['SYSTEM']).shift(t)[lmilk]
    min_lagt = pd.concat((aucts, min_lagt), axis=1)
    data = pd.merge(data, min_lagt, how='left', on=auct_key, suffixes=('', '_miny%s'%(t)) )

year_max_lag = data.groupby(auct_key, as_index=False).max().sort_values(['SYSTEM','YEAR','MONTH','DAY'])
for t in range(1,1+lags):
    max_lagt = year_max_lag.groupby(['SYSTEM']).shift(t)[lmilk]
    max_lagt = pd.concat((aucts, max_lagt), axis=1)
    data = pd.merge(data, max_lagt, how='left', on=auct_key, suffixes=('', '_maxy%s'%(t)) ) 
    

#pre processing to help fmozones
fe = ['FMOZONE']
data.loc[(data['FMOZONE'] =='1A') , 'FMOZONE'] = '1'

fekeys = []
for effect in fe:
    fes = pd.get_dummies(data[effect], drop_first=True)
    fekeys = fekeys+ list(fes.columns)
    data = pd.concat((data, fes), axis=1)

    
bid_key = auct_key + ['VENDOR'] + ['COUNTY']
covariates = lcts + dummies + fekeys

#setup interaction term for punishment
data['min*past'] = data['LSCORE_min1']*data['PAST_AUCT']

In [4]:
#write this to csvs
reg0 = data.copy()[bid_key + lmilk + covariates + year_lagkeys + lagkeys + ['INC','PAST_AUCT', 'min*past']]
reg0.to_csv('data/clean_milk0.csv')

In [5]:
#only drop data 5 periods back
maxlag = 4
limitedlag = [l+str(i) for l in ['LSCORE_min','LSCORE_max'] for i in range(1,1+maxlag)]


maxlagy = 2
limitedlagy = [l+str(i) for l in ['LSCORE_miny','LSCORE_maxy'] for i in range(1,1+maxlagy)]


limitedhist = ['PAST_AUCT','min*past'] + limitedlag + limitedlagy 


reg1 = data.copy()[bid_key + lmilk + covariates + limitedhist + ['INC','WIN']]
reg1 = reg1.dropna()
reg1.to_csv('data/clean_milk1.csv')

In [6]:
#baseline
sm.OLS(reg1['LSCORE'], sm.add_constant(reg1[covariates] ) ).fit().summary()

/home/erichschulman/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 LSCORE   R-squared:                       0.166
Model:                            OLS   Adj. R-squared:                  0.163
Method:                 Least Squares   F-statistic:                     56.80
Date:                Mon, 22 Jul 2019   Prob (F-statistic):          2.60e-115
Time:                        19:05:58   Log-Likelihood:                 3183.8
No. Observations:                3153   AIC:                            -6344.
Df Residuals:                    3141   BIC:                            -6271.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.2682      0.085    -26.614      0.000      -2.435      -2.101
LFMO           0.2227      0.030      7.305      0.000       0.163       0.283
LGAS           0.0090      0.005      1.648      0.099      -0.002       0.020
LPOPUL         0.0159      0.004      3.853      0.000       0.008       0.024
LQSCORE       -0.0213      0.004     -4.938      0.000      -0.030      -0.013
COOLER         0.0177      0.004      4.984      0.000       0.011       0.025
ESC           -0.0294      0.003     -8.895      0.000      -0.036      -0.023
NUM            0.0078      0.002      4.709      0.000       0.005       0.011
3             -0.0697      0.005    -13.592      0.000      -0.080      -0.060
6             -0.0413      0.015     -2.686      0.007      -0.072      -0.011
7             -0.1068      0.017     -6.417      0.000      -0.139      -0.074
9             -0.0520      0.005    -11.076      0.000      -0.061      -0.043
==============================================================================
Omnibus:                      268.816   Durbin-Watson:                   1.198
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1444.596
Skew:                           0.201   Prob(JB):                         0.00
Kurtosis:                       6.291   Cond. No.                         946.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
#incumbency
sm.OLS(reg1['LSCORE'], sm.add_constant(reg1[covariates+ ['INC']] ) ).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 LSCORE   R-squared:                       0.183
Model:                            OLS   Adj. R-squared:                  0.180
Method:                 Least Squares   F-statistic:                     58.78
Date:                Mon, 22 Jul 2019   Prob (F-statistic):          1.11e-128
Time:                        19:05:58   Log-Likelihood:                 3217.2
No. Observations:                3153   AIC:                            -6408.
Df Residuals:                    3140   BIC:                            -6330.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.2384      0.084    -26.515      0.000      -2.404      -2.073
LFMO           0.2168      0.030      7.181      0.000       0.158       0.276
LGAS           0.0094      0.005      1.749      0.080      -0.001       0.020
LPOPUL         0.0154      0.004      3.777      0.000       0.007       0.023
LQSCORE       -0.0208      0.004     -4.872      0.000      -0.029      -0.012
COOLER         0.0174      0.004      4.928      0.000       0.010       0.024
ESC           -0.0304      0.003     -9.284      0.000      -0.037      -0.024
NUM            0.0053      0.002      3.172      0.002       0.002       0.009
3             -0.0708      0.005    -13.947      0.000      -0.081      -0.061
6             -0.0409      0.015     -2.689      0.007      -0.071      -0.011
7             -0.1064      0.016     -6.457      0.000      -0.139      -0.074
9             -0.0540      0.005    -11.600      0.000      -0.063      -0.045
INC           -0.0306      0.004     -8.207      0.000      -0.038      -0.023
==============================================================================
Omnibus:                      285.446   Durbin-Watson:                   1.101
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1685.007
Skew:                           0.193   Prob(JB):                         0.00
Kurtosis:                       6.560   Cond. No.                         947.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
#2 lines incumbency

In [9]:
#previous prices 1 day
sm.OLS(reg1['LSCORE'], sm.add_constant(reg1[covariates + limitedlag]) ).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 LSCORE   R-squared:                       0.202
Model:                            OLS   Adj. R-squared:                  0.197
Method:                 Least Squares   F-statistic:                     41.69
Date:                Mon, 22 Jul 2019   Prob (F-statistic):          6.81e-138
Time:                        19:05:58   Log-Likelihood:                 3253.1
No. Observations:                3153   AIC:                            -6466.
Df Residuals:                    3133   BIC:                            -6345.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -1.4875      0.118    -12.639      0.000      -1.718      -1.257
LFMO            0.1519      0.031      4.931      0.000       0.092       0.212
LGAS            0.0109      0.005      2.032      0.042       0.000       0.021
LPOPUL          0.0097      0.004      2.375      0.018       0.002       0.018
LQSCORE        -0.0176      0.004     -4.139      0.000      -0.026      -0.009
COOLER          0.0168      0.003      4.795      0.000       0.010       0.024
ESC            -0.0277      0.003     -8.509      0.000      -0.034      -0.021
NUM             0.0063      0.002      3.819      0.000       0.003       0.009
3              -0.0715      0.005    -14.186      0.000      -0.081      -0.062
6              -0.0443      0.015     -2.915      0.004      -0.074      -0.015
7              -0.1084      0.016     -6.607      0.000      -0.141      -0.076
9              -0.0572      0.005    -12.325      0.000      -0.066      -0.048
LSCORE_min1    -0.0549      0.020     -2.790      0.005      -0.093      -0.016
LSCORE_min2     0.0278      0.020      1.371      0.171      -0.012       0.067
LSCORE_min3     0.0675      0.020      3.342      0.001       0.028       0.107
LSCORE_min4     0.0037      0.020      0.181      0.856      -0.036       0.043
LSCORE_max1     0.1498      0.021      7.133      0.000       0.109       0.191
LSCORE_max2     0.0497      0.021      2.334      0.020       0.008       0.091
LSCORE_max3     0.0582      0.021      2.817      0.005       0.018       0.099
LSCORE_max4     0.0340      0.020      1.717      0.086      -0.005       0.073
==============================================================================
Omnibus:                      318.204   Durbin-Watson:                   1.260
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1731.051
Skew:                           0.319   Prob(JB):                         0.00
Kurtosis:                       6.573   Cond. No.                     1.35e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.35e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [10]:
#prev prices with prev auction
sm.OLS(reg1['LSCORE'], sm.add_constant(reg1[covariates + ['PAST_AUCT','min*past'] + limitedlag ]) ).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 LSCORE   R-squared:                       0.204
Model:                            OLS   Adj. R-squared:                  0.199
Method:                 Least Squares   F-statistic:                     38.20
Date:                Mon, 22 Jul 2019   Prob (F-statistic):          4.57e-138
Time:                        19:05:58   Log-Likelihood:                 3257.4
No. Observations:                3153   AIC:                            -6471.
Df Residuals:                    3131   BIC:                            -6338.
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -1.5475      0.120    -12.894      0.000      -1.783      -1.312
LFMO            0.1481      0.031      4.805      0.000       0.088       0.209
LGAS            0.0110      0.005      2.059      0.040       0.001       0.022
LPOPUL          0.0095      0.004      2.340      0.019       0.002       0.018
LQSCORE        -0.0176      0.004     -4.134      0.000      -0.026      -0.009
COOLER          0.0162      0.003      4.630      0.000       0.009       0.023
ESC            -0.0277      0.003     -8.529      0.000      -0.034      -0.021
NUM             0.0062      0.002      3.777      0.000       0.003       0.009
3              -0.0718      0.005    -14.213      0.000      -0.082      -0.062
6              -0.0432      0.015     -2.839      0.005      -0.073      -0.013
7              -0.1082      0.016     -6.596      0.000      -0.140      -0.076
9              -0.0568      0.005    -12.232      0.000      -0.066      -0.048
PAST_AUCT       0.1684      0.063      2.679      0.007       0.045       0.292
min*past        0.0936      0.034      2.733      0.006       0.026       0.161
LSCORE_min1    -0.1014      0.025     -3.981      0.000      -0.151      -0.051
LSCORE_min2     0.0272      0.020      1.347      0.178      -0.012       0.067
LSCORE_min3     0.0680      0.020      3.367      0.001       0.028       0.108
LSCORE_min4     0.0025      0.020      0.125      0.901      -0.037       0.042
LSCORE_max1     0.1580      0.021      7.414      0.000       0.116       0.200
LSCORE_max2     0.0504      0.021      2.369      0.018       0.009       0.092
LSCORE_max3     0.0580      0.021      2.805      0.005       0.017       0.098
LSCORE_max4     0.0337      0.020      1.702      0.089      -0.005       0.073
==============================================================================
Omnibus:                      313.516   Durbin-Watson:                   1.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1693.870
Skew:                           0.312   Prob(JB):                         0.00
Kurtosis:                       6.536   Cond. No.                     1.40e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
#previous prices prev year
sm.OLS(reg1['LSCORE'], sm.add_constant(reg1[covariates+limitedhist]) ).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 LSCORE   R-squared:                       0.417
Model:                            OLS   Adj. R-squared:                  0.412
Method:                 Least Squares   F-statistic:                     89.35
Date:                Mon, 22 Jul 2019   Prob (F-statistic):               0.00
Time:                        19:05:58   Log-Likelihood:                 3747.5
No. Observations:                3153   AIC:                            -7443.
Df Residuals:                    3127   BIC:                            -7286.
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -1.0017      0.106     -9.428      0.000      -1.210      -0.793
LFMO             0.3218      0.027     11.882      0.000       0.269       0.375
LGAS            -0.0162      0.005     -3.464      0.001      -0.025      -0.007
LPOPUL           0.0086      0.003      2.459      0.014       0.002       0.015
LQSCORE         -0.0144      0.004     -3.929      0.000      -0.022      -0.007
COOLER           0.0011      0.003      0.364      0.716      -0.005       0.007
ESC             -0.0194      0.003     -6.920      0.000      -0.025      -0.014
NUM              0.0072      0.001      5.097      0.000       0.004       0.010
3               -0.0301      0.005     -6.597      0.000      -0.039      -0.021
6               -0.0304      0.013     -2.331      0.020      -0.056      -0.005
7               -0.0136      0.014     -0.940      0.347      -0.042       0.015
9               -0.0108      0.004     -2.546      0.011      -0.019      -0.002
PAST_AUCT        0.1215      0.054      2.254      0.024       0.016       0.227
min*past         0.0686      0.029      2.335      0.020       0.011       0.126
LSCORE_min1     -0.0874      0.022     -4.005      0.000      -0.130      -0.045
LSCORE_min2      0.0425      0.017      2.439      0.015       0.008       0.077
LSCORE_min3      0.0152      0.017      0.876      0.381      -0.019       0.049
LSCORE_min4      0.0211      0.017      1.211      0.226      -0.013       0.055
LSCORE_max1      0.1024      0.018      5.575      0.000       0.066       0.138
LSCORE_max2      0.0212      0.018      1.158      0.247      -0.015       0.057
LSCORE_max3      0.0360      0.018      2.030      0.042       0.001       0.071
LSCORE_max4      0.0415      0.017      2.445      0.015       0.008       0.075
LSCORE_miny1     0.4748      0.024     19.587      0.000       0.427       0.522
LSCORE_miny2     0.0252      0.025      1.019      0.308      -0.023       0.074
LSCORE_maxy1     0.1255      0.021      6.045      0.000       0.085       0.166
LSCORE_maxy2     0.0163      0.020      0.811      0.417      -0.023       0.056
==============================================================================
Omnibus:                      325.030   Durbin-Watson:                   1.669
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1835.210
Skew:                           0.318   Prob(JB):                         0.00
Kurtosis:                       6.683   Cond. No.                     1.47e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.47e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""